In [2]:
def augmation(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        
        #targetが1のものだけコピー
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder


In [9]:

#X=2次元train_data,出力時に整形して1次元にする
class KMeansFeaturizer:
    """
    数値データをk-meansのクラスタIDに変換します。
    
    この変換器は入力データに対してk-meansを実行し、各データポイントを最も近い
    クラスタのIDに変換します。ターゲット変数yが存在する場合、クラス分類の境界を
    より重視したクラスタリングの結果を得るために、ターゲット変数をスケーリングして
    入力データに含めてk-meansに渡します。
    """
    #k=出力するクラスタの数
    def __init__(self, k=100, target_scale=5.0, random_state=None):
        self.k = k
        self.target_scale = target_scale
        self.random_state = random_state
        # np.array(range(k)).reshape(-1,1) ->2次元配列にそれぞれ一個ずつ要素が入っている。([[0],[1]...])
        #要素が入ってきたとき、one-hotvectorに変換する?([0]->[1,0,0...])
        self.cluster_encoder = OneHotEncoder(categories='auto').fit(np.array(range(k)).reshape(-1,1))
        
    def fit(self, X, y=None):
        """
        入力データに対しk-meansを実行し、各クラスタの中心を見つけます。
        """
        if y is None:
            # ターゲット変数がない場合、ふつうのk-meansを実行します。
            km_model = KMeans(n_clusters=self.k, n_init=20, random_state=self.random_state)
            km_model.fit(X)
            self.km_model = km_model
            self.cluster_centers_ = km_model.cluster_centers_
            return self

        # ターゲット変数がある場合。スケーリングして入力データに含めます。
        data_with_target = np.hstack((X, y[:,np.newaxis]*self.target_scale))

        # ターゲットを組み入れたデータで事前学習するためのk-meansモデルを構築します。
        km_model_pretrain = KMeans(n_clusters=self.k, n_init=20, random_state=self.random_state)
        km_model_pretrain.fit(data_with_target)

        # ターゲット変数の情報を除いて元の空間におけるクラスタを得るために
        # k-meansを再度実行します。事前学習で見つけたクラスタの中心を
        # 使って初期化し、クラスタの割り当てと中心の再計算を1回にします。
        km_model = KMeans(n_clusters=self.k, init=km_model_pretrain.cluster_centers_[:,:2], n_init=1, max_iter=1)
        km_model.fit(X)
        
        self.km_model = km_model
        self.cluster_centers_ = km_model.cluster_centers_
        return self
    
    def transform(self, X, y=None):
        """
        入力データポイントに最も近いクラスタのIDを返します。
        """
        clusters = self.km_model.predict(X)
        #clustersが[[0],[8]...]みたいに代入されてそれがワンホットベクトルとして変換されて出力される?
        return self.cluster_encoder.transform(clusters.reshape(-1,1))
    
    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [3]:
import feather as ftr
train = ftr.read_dataframe('../data/input/train.feather')
test = ftr.read_dataframe('../data/input/test.feather')


In [20]:
%%time
# train.drop(columns=['target','ID_code'],inplace=True)
train=train.iloc[:10000,:]
# kmf=KMeansFeaturizer()
kmf.fit(train)

Wall time: 40.3 s


In [21]:
%%time
k_train=kmf.transform(train)

Wall time: 44.9 ms


In [19]:
import pandas as pd
df=pd.DataFrame(k_train.toarray())
df=df.add_prefix('col_')
df.head(100)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_90,col_91,col_92,col_93,col_94,col_95,col_96,col_97,col_98,col_99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
kmf.__dict__

{'k': 100,
 'target_scale': 5.0,
 'random_state': None,
 'cluster_encoder': OneHotEncoder(categorical_features=None, categories='auto',
        dtype=<class 'numpy.float64'>, handle_unknown='error',
        n_values=None, sparse=True),
 'km_model_': KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
     n_clusters=100, n_init=20, n_jobs=None, precompute_distances='auto',
     random_state=None, tol=0.0001, verbose=0),
 'cluster_centers_': array([[10.21833571, -1.83855571, 11.11872143, ...,  8.84772571,
         15.89844286, -1.09977857],
        [10.74640612, -1.52088163, 10.3938449 , ...,  9.04844184,
         15.67786939,  0.64786327],
        [10.6091725 , -1.47059167, 10.63091   , ...,  8.8554325 ,
         15.92952917, -2.2190425 ],
        ...,
        [10.87314854, -1.42585243, 10.43641748, ...,  9.01350194,
         15.65016699, -2.95694757],
        [10.52480286, -1.24864357, 10.78198643, ...,  8.86920786,
         16.202675  , -3.44614786],
        [10.617